# Setup Lengkap

In [ ]:
# @title 👣 Buat struktur folder
import os

base_dir = "/content/transkrip_project"
video_dir = os.path.join(base_dir, "video")
audio_dir = os.path.join(base_dir, "audio")
output_dir = os.path.join(base_dir, "output")
log_dir = os.path.join(base_dir, "logs")

os.makedirs(video_dir, exist_ok=True)
os.makedirs(audio_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

print("📁 Struktur folder dibuat:")
print(f"- Video: {video_dir}")
print(f"- Audio: {audio_dir}")
print(f"- Output: {output_dir}")
print(f"- Log: {log_dir}")

In [2]:
# @title 🧩 Install Library Tambahan
!pip install -q gdown
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q yt-dlp
!pip install -q tqdm
!pip install -q ffmpeg-python
!pip install -q m3u8downloader
!sudo apt-get update -y
!sudo apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.6 MB/s eta 0:00:00


# Download Video

In [ ]:
# @title 🍪 [Manual] Upload Cookies
# @markdown - Upload file cookies (Untuk download twitter yang membutuhkan auth)
from google.colab import files
import os

print("📤 Silakan upload file cookies.txt (hasil dari Kiwi Browser)...")
uploaded = files.upload()

# Temukan file .txt yang diupload
cookies_file = None
for name in uploaded.keys():
    if name.endswith(".txt"):
        cookies_file = name
        break

# Rename ke cookies.txt
if cookies_file and os.path.exists(cookies_file):
    if cookies_file != "cookies.txt":
        os.rename(cookies_file, "cookies.txt")
        print(f"📁 File '{cookies_file}' di-rename menjadi 'cookies.txt'")
    else:
        print("✅ File sudah bernama cookies.txt")

    # Tampilkan contoh isi file
    print("📄 Contoh isi cookies.txt:")
    with open("cookies.txt", "r", encoding="utf-8") as f:
        for i in range(3):
            line = f.readline()
            if line:
                print("  ", line.strip())
else:
    print("❌ File .txt tidak valid ditemukan atau gagal upload.")

In [ ]:
# @title 🔗 [A1] Form Input: Masukkan Link, Pilih Alat, dan Nama File
# @markdown - nama file boleh kosong
download_type = "auto"  # @param ["auto", "google_drive", "direct", "m3u8"]
video_url = ""  # @param {type:"string"}
file_name = ""  # @param {type:"string"}

from datetime import datetime
import os

# 🔧 Lokasi penyimpanan
video_dir = "/content/transkrip_project/video"
os.makedirs(video_dir, exist_ok=True)

# 🕒 Nama file default jika kosong
if not file_name.strip():
    timestamp = datetime.now().strftime("video_%Y%m%d_%H%M%S")
    file_name = f"{timestamp}.mp4"
elif not file_name.endswith(".mp4"):
    file_name += ".mp4"

# 📍 Path output final
output_path = os.path.join(video_dir, file_name)

print(f"🎯 Link: {video_url}")
print(f"🧩 Jenis Unduhan: {download_type}")
print(f"📁 File akan disimpan di: {output_path}")

In [ ]:
# @title 🚀 [A2] Proses Download Berdasarkan Jenis yang Dipilih
# @markdown 👈 klik untuk mengunduh

import subprocess
import re
import os

def extract_drive_id(url):
    patterns = [
        r"drive\.google\.com\/file\/d\/([^\/]+)",
        r"drive\.google\.com\/open\?id=([^&]+)",
        r"drive\.google\.com\/uc\?id=([^&]+)",
    ]
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def is_m3u8(url):
    return url.endswith(".m3u8") or ".m3u8?" in url

def is_drive(url):
    return "drive.google.com" in url

# 🔧 Pastikan dependensi
print("🔧 Memastikan dependensi...")
!apt -qq install -y ffmpeg
!pip install -q --upgrade yt-dlp
!pip install -q --user gdown m3u8downloader

# 🌐 Tentukan alat berdasarkan input user atau auto-detect
tool = download_type

if tool == "auto":
    if is_drive(video_url):
        tool = "google_drive"
    elif is_m3u8(video_url):
        tool = "m3u8"
    else:
        tool = "direct"

print(f"🚀 Menggunakan alat: {tool}")

# ===================== Proses Unduh =====================
if tool == "google_drive":
    print("📁 Mendownload dari Google Drive...")
    import gdown
    drive_id = extract_drive_id(video_url)
    if not drive_id:
        raise ValueError("❌ Tidak dapat mendeteksi ID file dari URL Google Drive.")
    gdown.download(f"https://drive.google.com/uc?id={drive_id}", output_path, quiet=False)

elif tool == "m3u8":
    print("📺 Mendownload dari M3U8 (streaming)...")
    cmd = [
        os.path.expanduser("~/.local/bin/downloadm3u8"),
        "-o", output_path,
        video_url
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in process.stdout:
        print(f"\r{line.strip()[:150]}", end="", flush=True)
    process.wait()

elif tool == "direct":
    print("📥 Mendownload dari Direct Link menggunakan yt-dlp...")
    cmd = [
        "yt-dlp",
        "-f", "best",
        "-o", output_path,
        "--no-warnings",
        "--retries", "3",
        video_url
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in process.stdout:
        print(f"\r{line.strip()[:150]}", end="", flush=True)
    process.wait()

else:
    raise ValueError("❌ Jenis download tidak dikenali!")

# ✅ Cek hasil
if os.path.exists(output_path):
    size = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n\n✅ Selesai! File disimpan di: {output_path}")
    print(f"📦 Ukuran file: {size:.2f} MB")
else:
    print("\n⚠️ Download selesai tapi file tidak ditemukan.")

In [ ]:
# @title 🎞️ [B] Unduh Video Twitter (Jumlah Otomatis)
# @markdown - Masukkan URL tweet (multi-video / carousel)
tweet_url = ""  # @param {type: "string"}

import os
import re
import json
import glob
import subprocess

# 📁 Siapkan folder
base_dir = "/content/transkrip_project"
video_dir = os.path.join(base_dir, "video")
output_dir = os.path.join(base_dir, "output")
os.makedirs(video_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# 🆔 Ambil Tweet ID
match = re.search(r"/status/(\d+)", tweet_url)
if not match:
    raise ValueError("❌ URL tidak valid: Tidak ditemukan Tweet ID.")
tweet_id = match.group(1)

# 🔍 Simulasi metadata (jika memungkinkan)
info = None
ydl_opts = {
    "quiet": True,
    "simulate": True,
    "extract_flat": True,
    "dump_single_json": True,
}

print(f"🔍 Mendeteksi metadata tweet... ({tweet_url})")

try:
    import yt_dlp
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(tweet_url, download=False)
except Exception as e:
    print(f"❌ Gagal mendeteksi metadata: {e}")
    if "authentication" in str(e).lower():
        print("🔐 Tweet ini kemungkinan membutuhkan cookies.txt (login).")

# 💾 Simpan JSON jika tersedia
if info:
    json_path = os.path.join(output_dir, f"deteksi_tweet_{tweet_id}.json")
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(info, f, ensure_ascii=False, indent=2)
    print(f"📄 Metadata disimpan ke: {json_path}")
else:
    print("⚠️ Metadata tidak tersedia atau tidak disimpan.")

# 🍪 Cek cookies
use_cookies = os.path.exists("cookies.txt")
print("🔐 Menggunakan cookies.txt" if use_cookies else "🔓 Tidak menggunakan cookies")

# 📥 Mulai proses download
print("📥 Mulai mengunduh video...\n")

# Buat command yt-dlp
command = ["yt-dlp"]
if use_cookies:
    command += ["--cookies", "cookies.txt"]
command += [
    "-f", "best",
    "-o", f"{video_dir}/%(id)s_video.%(ext)s",
    tweet_url
]

# Jalankan dan tampilkan log secara realtime
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

for line in process.stdout:
    print(line, end="")  # log baris per baris

process.wait()

# 📦 Cek hasil download
downloaded = glob.glob(os.path.join(video_dir, "*_video.*"))
print(f"\n✅ Semua video selesai diunduh ke: {video_dir}")
print(f"🎞️ Total video yang berhasil diunduh: {len(downloaded)} file.")

# Ekstrak Audio dari Video

In [ ]:
# @title 🔉 Ekstrak Audio
# @markdown - Masukkan file Videonya
video_input_path = ""  # @param {type:"string"}

import os

# Folder output audio
audio_dir = "/content/transkrip_project/audio"
os.makedirs(audio_dir, exist_ok=True)

# Ambil nama file video
video_name = os.path.splitext(os.path.basename(video_input_path))[0]
audio_path = os.path.join(audio_dir, f"{video_name}.wav")

# Fungsi konversi ukuran file
def human_readable_size(size_bytes):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024
    return f"{size_bytes:.2f} PB"

# Hapus file audio lama jika sudah ada
if os.path.exists(audio_path):
    os.remove(audio_path)

# Eksekusi ekstraksi
print("🔊 Menjalankan ekstraksi audio...")
print(f"🎬 Input:  {video_input_path}")
print(f"🎧 Output: {audio_path}")

!ffmpeg -i "{video_input_path}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{audio_path}" -y -loglevel error

# Konfirmasi hasil
if os.path.exists(audio_path):
    size_str = human_readable_size(os.path.getsize(audio_path))
    print("\n✅ Audio berhasil diekstrak!")
    print(f"📍 Lokasi  : {audio_path}")
    print(f"📦 Ukuran  : {size_str}")
else:
    print("\n❌ Gagal mengekstrak audio.")

# Video to Teks

In [ ]:
# @title 👣 [1] Pilih Model Whisper { run: "auto" }
model_choice = "base"  # @param ["tiny", "base", "small", "medium", "large"]

print(f"Model : {model_choice}")

Model : base


In [ ]:
# @title 👣 [2] Transkripsi Audio
# @markdown - Gunakan Model dari Cell 1

# @markdown - Otomatis menyimpan hasil transkripsi ke `/content/transkrip_project/output`

audio_input_path = "/content/transkrip_project/audio/PRED-144 Eimi Fukada.wav"  # @param {type:"string"}

import whisper
import time
import os
import json

# Lokasi folder output
output_dir = "/content/transkrip_project/output"
os.makedirs(output_dir, exist_ok=True)

# Validasi audio path
if not os.path.exists(audio_input_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {audio_input_path}")

# Gunakan model dari cell sebelumnya
chosen_model = model_choice
print(f"📚 Memuat model Whisper: {chosen_model} ...")
start = time.time()

model = whisper.load_model(chosen_model)
result = model.transcribe(audio_input_path, verbose=True)

end = time.time()
print(f"⏱️ Transkripsi selesai dalam {end - start:.2f} detik")

# 📄 Simpan hasil segments ke JSON
json_output_path = os.path.join(output_dir, "segments.json")
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(result["segments"], f, ensure_ascii=False, indent=2)

# 📝 Cuplikan hasil
print(f"\n✅ Hasil transkripsi disimpan ke: {json_output_path}")
print("📝 Cuplikan teks:")
print(result["text"][:100] + "...")

In [ ]:
# @title 👣 [3] Ekspor ke Subtitle .srt dengan Auto-Split Teks { run: "auto" }
# @markdown - Atur maksimal jumlah karakter per baris subtitle
json_path = "/content/transkrip_project/output/segments.json"  # @param {type:"string"}
max_chars_per_line = 95  # @param {type:"slider", min:40, max:120, step:5}

import os
import json
import textwrap
from datetime import timedelta

# 🔧 Fungsi bantu konversi waktu
def format_time(seconds):
    t = timedelta(seconds=seconds)
    return str(t).split(".")[0].zfill(8).replace(".", ",") + f",{int(t.microseconds/1000):03d}"

# 🔠 Fungsi bungkus teks per baris
def wrap_text(text, width=80):
    return "\n".join(textwrap.wrap(text, width=width))

# 📥 Baca file JSON transkrip
if not os.path.exists(json_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {json_path}")

with open(json_path, "r", encoding="utf-8") as f:
    segments = json.load(f)

# 📝 Buat konten subtitle
srt_lines = []
for i, seg in enumerate(segments, 1):
    start = format_time(seg["start"])
    end = format_time(seg["end"])
    text = wrap_text(seg["text"].strip(), width=max_chars_per_line)

    srt_lines.append(f"{i}")
    srt_lines.append(f"{start} --> {end}")
    srt_lines.append(text)
    srt_lines.append("")

# 💾 Simpan ke file .srt dengan nama sama seperti JSON
json_base_name = os.path.splitext(os.path.basename(json_path))[0]
srt_output_path = os.path.join("/content/transkrip_project/output", f"{json_base_name}.srt")

with open(srt_output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(srt_lines))

print(f"✅ Subtitle disimpan ke: {srt_output_path}")

# Upload to Gogle Drive

In [ ]:
# @title 🔌 [Mount] Sambungkan Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title ☁️ [Manual] Upload ke Google Drive
# @markdown - Masukkan path folder tujuan di Google Drive
drive_target_folder = ""  # @param {type:"string"}

# @markdown - Masukkan nama file di Google Drive
drive_filename = ""  # @param {type:"string"}

# @markdown - Masukkan file lokal yang ingin diupload
file_to_upload = ""  # @param {type:"string"}

import os, shutil

# Cek apakah Google Drive sudah di-mount
if not os.path.exists("/content/drive"):
    print("❌ Google Drive belum di-mount. Jalankan mount terlebih dahulu!")
else:
    # Bangun path lengkap tujuan
    drive_target_path = os.path.join(drive_target_folder, drive_filename)

    # Validasi file sumber
    if not os.path.exists(file_to_upload):
        print("❌ File sumber tidak ditemukan.")
    elif not drive_target_path.startswith("/content/drive/"):
        print("❌ Path tujuan harus berada di dalam /content/drive/")
    else:
        # Buat folder tujuan jika belum ada
        if not os.path.exists(drive_target_folder):
            os.makedirs(drive_target_folder)
            print(f"📂 Folder baru dibuat: {drive_target_folder}")

        # Salin file
        shutil.copy(file_to_upload, drive_target_path)
        print(f"✅ File berhasil dikirim ke Google Drive: {drive_target_path}")

# Utility

In [ ]:
# @title 🔍 Deteksi Apakah Audio Lossless atau Lossy
# @markdown - Masukkan path lengkap file audio:
audio_file_path = ""  # @param {type:"string"}

import subprocess
import json
import os

def is_lossless(codec_name):
    # Daftar codec yang umumnya lossless
    lossless_codecs = [
        "flac", "pcm_s16le", "pcm_s24le", "alac", "ape", "wavpack", "tta", "mlp"
    ]
    return codec_name in lossless_codecs

if not os.path.exists(audio_file_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {audio_file_path}")

# Jalankan ffprobe untuk dapatkan metadata
cmd = [
    "ffprobe", "-v", "error", "-show_streams", "-print_format", "json", audio_file_path
]
result = subprocess.run(cmd, capture_output=True, text=True)
info = json.loads(result.stdout)

# Ambil stream audio pertama
audio_stream = next((stream for stream in info["streams"] if stream["codec_type"] == "audio"), None)

if audio_stream:
    codec = audio_stream.get("codec_name", "unknown")
    sample_rate = audio_stream.get("sample_rate", "unknown")
    channels = audio_stream.get("channels", "unknown")
    bit_rate = int(audio_stream.get("bit_rate", 0)) if "bit_rate" in audio_stream else None
    duration = float(audio_stream.get("duration", 0.0))

    readable_rate = f"{int(bit_rate)/1000:.1f} kbps" if bit_rate else "Unknown"

    print(f"📄 Informasi Audio:")
    print(f"🔹 Codec       : {codec}")
    print(f"🔹 Sample Rate : {sample_rate} Hz")
    print(f"🔹 Channels    : {channels}")
    print(f"🔹 Bitrate     : {readable_rate}")
    print(f"⏱️ Durasi      : {duration:.2f} detik")

    if is_lossless(codec):
        print("\n✅ Audio ini menggunakan codec lossless.")
    else:
        print("\n⚠️ Audio ini tampaknya **lossy** (kompresi dengan kehilangan data).")
else:
    print("❌ Tidak ada stream audio yang ditemukan dalam file.")

In [ ]:
# @title 📑 Tampilkan Ringkasan Transkripsi

# @markdown - Menampilkan Ringkasan dari File JSON atau SRT

transcript_path = ""  # @param {type:"string"}

import os
import json

# Validasi file
if not os.path.exists(transcript_path):
    raise FileNotFoundError(f"❌ File tidak ditemukan: {transcript_path}")

print(f"📑 Menampilkan isi dari: {transcript_path}\n")

# ========= Jika .json (format Whisper) =========
if transcript_path.endswith(".json"):
    with open(transcript_path, "r", encoding="utf-8") as f:
        segments = json.load(f)

    for seg in segments:
        start = round(seg["start"], 2)
        end = round(seg["end"], 2)
        text = seg["text"].strip()
        print(f"[{start} - {end}] {text}")

# ========= Jika .srt (subtitle biasa) =========
elif transcript_path.endswith(".srt"):
    with open(transcript_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    block = []
    for line in lines:
        line = line.strip()
        if line == "":
            if len(block) >= 3:
                index = block[0]
                timecode = block[1]
                text = " ".join(block[2:])
                print(f"[{timecode}] {text}")
            block = []
        else:
            block.append(line)

else:
    raise ValueError("❌ Format file tidak didukung. Gunakan .json atau .srt.")

In [ ]:
# @title 🎬 [Insert] Embed Subtitle (.srt) → Output MKV
# @markdown - Masukkan path lengkap video dan subtitle (.srt)
video_input_path = ""  # @param {type:"string"}
subtitle_path = ""  # @param {type:"string"}

import os
import subprocess

# Ambil nama asli video tanpa ekstensi
video_basename = os.path.splitext(os.path.basename(video_input_path))[0]

# Output video (mkv)
output_video_path = os.path.join(output_dir, f"{video_basename}_with_subtitles.mkv")

# Jalankan ffmpeg
cmd = [
    "ffmpeg",
    "-i", video_input_path,
    "-i", subtitle_path,
    "-c", "copy",        # tanpa re-encode
    "-c:s", "srt",       # jaga format subtitle .srt
    "-map", "0",
    "-map", "1",
    output_video_path
]

print("🎞️ Menyisipkan subtitle ke video (.mkv, softsub)...")
subprocess.run(cmd, check=True)
print(f"✅ Video akhir disimpan di: {output_video_path}")

In [ ]:
# @title 🗂️ [Manual] Kompres Folder Menjadi .zip
# @markdown - Masukkan path folder dan nama file zip output
folder_to_zip = ""  # @param {type:"string"}
zip_filename = ""  # @param {type:"string"}

import shutil
import os

# Lokasi penyimpanan ZIP di dalam transkrip_project
zip_path = os.path.join("/content/transkrip_project", zip_filename)

# Validasi folder input
if not os.path.exists(folder_to_zip):
    raise FileNotFoundError(f"❌ Folder tidak ditemukan: {folder_to_zip}")

# Kompres folder
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_to_zip)
print(f"✅ Folder berhasil dikompres ke ZIP: {zip_path}")

# Tampilkan isi folder yang dikompres
print("\n📂 Daftar isi dalam folder:")
for root, dirs, files in os.walk(folder_to_zip):
    for file in files:
        rel_dir = os.path.relpath(root, folder_to_zip)
        rel_file = os.path.join(rel_dir, file) if rel_dir != "." else file
        print(f"  ├─ {rel_file}")

# Tampilkan ukuran file zip
if os.path.exists(zip_path):
    size_mb = os.path.getsize(zip_path) / (1024 * 1024)
    print(f"\n📦 Ukuran file ZIP: {size_mb:.2f} MB")
else:
    print("⚠️ File ZIP tidak ditemukan.")

In [ ]:
# @title 📤 [Manual] Cek & Ekstrak Subtitle Softsub dari Video
# @markdown - Masukkan path video yang ingin dicek
video_with_sub_path = ""  # @param {type:"string"}

import subprocess, os, json

# Lokasi output subtitle
subtitle_extracted_path = os.path.join(output_dir, "extracted_subtitle.srt")

# 1️⃣ Cek dengan ffprobe apakah subtitle stream tersedia
probe_cmd = [
    "ffprobe",
    "-v", "error",
    "-select_streams", "s",
    "-show_entries", "stream=index:stream_tags=language",
    "-of", "json",
    video_with_sub_path
]

probe_result = subprocess.run(probe_cmd, capture_output=True, text=True)

# Parse hasil JSON
streams = json.loads(probe_result.stdout).get("streams", [])

if not streams:
    print("⚠️ Video tidak memiliki stream subtitle (softsub).")
else:
    print(f"✅ Ditemukan {len(streams)} subtitle stream. Mengekstrak stream pertama...")

    # 2️⃣ Jalankan ffmpeg untuk ekstrak subtitle pertama
    extract_cmd = [
        "ffmpeg",
        "-y",
        "-i", video_with_sub_path,
        "-map", "0:s:0",
        subtitle_extracted_path
    ]
    result = subprocess.run(extract_cmd, capture_output=True, text=True)

    if result.returncode == 0 and os.path.exists(subtitle_extracted_path):
        print(f"✅ Subtitle berhasil diekstrak ke: {subtitle_extracted_path}")
    else:
        print("❌ Gagal mengekstrak subtitle.")
        print("📄 FFmpeg log:")
        print(result.stderr)

In [ ]:
# @title 📄 [View Log] Tampilkan isi log yt-dlp
# @markdown - Menampilkan isi file yt_dlp_log.txt (log download video)

log_path = "/content/transkrip_project/logs/yt_dlp_log.txt"  # lokasi log

if os.path.exists(log_path):
    print(f"📁 Menampilkan isi log dari: {log_path}\n")

    with open(log_path, "r", encoding="utf-8", errors="ignore") as f:
        log_lines = f.readlines()

    # Tampilkan hanya 50 baris terakhir (ubah sesuai kebutuhan)
    num_lines = 50
    for line in log_lines[-num_lines:]:
        print(line.rstrip())
else:
    print("❌ File log belum tersedia atau belum dibuat.")

In [ ]:
# @title 🧹 Bersihkan File Setelah Proses
# @markdown Pilih folder yang ingin dibersihkan:
hapus_video = True  # @param {type:"boolean"}
hapus_audio = True  # @param {type:"boolean"}
hapus_output = True  # @param {type:"boolean"}
hapus_logs = True  # @param {type:"boolean"}

import os

def delete_all_files(folder):
    if not os.path.exists(folder):
        print(f"❌ Folder tidak ditemukan: {folder}")
        return

    deleted = False
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if os.path.isfile(file_path):
            try:
                os.remove(file_path)
                print(f"🗑️ Dihapus: {file_path}")
                deleted = True
            except Exception as e:
                print(f"⚠️ Gagal hapus {file_path}: {e}")
    if not deleted:
        print(f"ℹ️ Tidak ada file yang dihapus di: {folder}")

# Jalankan penghapusan per folder jika dicentang
if hapus_video:
    delete_all_files(video_dir)

if hapus_audio:
    delete_all_files(audio_dir)

if hapus_output:
    delete_all_files(output_dir)

if hapus_logs:
    delete_all_files(log_dir)